# Fibonacci sequence

The Fibonacci sequence is a series of numbers where a number is found by adding up the two numbers before it. Starting with 0 and 1, the sequence goes 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, and so forth. Written as a rule, the expression is:
## $$x_n = x_{n-1} + x_{n-2}$$
![alt text](https://landperspectives.files.wordpress.com/2014/12/1-fibonacci-sequence1.jpg )

***
## Defining python function

In [1]:
def fib(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a+b, a
    return a

In [2]:
%timeit fib(120)

10000 loops, best of 3: 27 µs per loop


## Using cython
The same function can be copied and pasted into cython. In fact, cython can compile (nearly) all python code.
```cython
def fib_cython_dynamic(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a+b, a
    return a
```

In [3]:
from fib_cython import fib_cython_dynamic

In [5]:
%timeit fib_cython_dynamic(120)

1000000 loops, best of 3: 1.33 µs per loop


A bit of performance boost, but that's not enough... Let's add static typesetting:
```cython
cpdef fib_cython_static(int n):
    cdef:
        double a=0.0, b=1.0
        int i;
    for i in range(n):
        a, b = a + b, a
    return a
```
Now that we typeset our function, cython can create optimized code, based on the data types.

In [6]:
from fib_cython import fib_cython_static

In [7]:
%timeit fib_cython_static(120)

The slowest run took 7.66 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 280 ns per loop


Very good up till now, but we can still do better. Normally python perform what's called offbounds and wraparound checking. We can disable that for our function by doing:
```cython
cimport cython
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef fib_cython_static(int n):
    if n<2:
        return n
    return fib_cython_static(n-2) + fib_cython_static(n-1)
```

In [8]:
from fib_cython import fib_cython_opt

In [9]:
%timeit fib_cython_opt(120)

The slowest run took 7.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 255 ns per loop
